<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


 ## Lab 2: Data wrangling 


Estimated time needed: **60** minutes


In this lab, we will perform some Exploratory Data Analysis (EDA) to find some patterns in the data and determine what would be the label for training supervised models. 

In the data set, there are several different cases where the booster did not land successfully. Sometimes a landing was attempted but failed due to an accident; for example, <code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully landed on  a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed on a drone ship. 

In this lab we will mainly convert those outcomes into Training Labels with `1` means the booster successfully landed `0` means it was unsuccessful.


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


## Objectives
Perform exploratory  Data Analysis and determine Training Labels 

- Exploratory Data Analysis
- Determine Training Labels 


----


Install the below libraries


In [2]:
!pip install pandas
!pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 132.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 169.6 MB/s eta 0:00:00


## Import Libraries and Define Auxiliary Functions


We will import the following libraries.


In [3]:
# Pandas is a software library written for the Python programming language for data manipulation and analysis.
import pandas as pd
#NumPy is a library for the Python programming language, adding support for large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays
import numpy as np

### Data Analysis 


Load Space X dataset, from last section.


In [4]:
df=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/dataset_part_1.csv")
df.head(10)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857
5,6,2014-01-06,Falcon 9,3325.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1005,-80.577366,28.561857
6,7,2014-04-18,Falcon 9,2296.000000,ISS,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1006,-80.577366,28.561857
7,8,2014-07-14,Falcon 9,1316.000000,LEO,CCAFS SLC 40,True Ocean,1,False,False,True,NaN,1.0,0,B1007,-80.577366,28.561857
8,9,2014-08-05,Falcon 9,4535.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1008,-80.577366,28.561857
9,10,2014-09-07,Falcon 9,4428.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1011,-80.577366,28.561857


Identify and calculate the percentage of the missing values in each attribute


In [5]:
df.isnull().sum()/len(df)*100

FlightNumber       0.000000
Date               0.000000
BoosterVersion     0.000000
PayloadMass        0.000000
Orbit              0.000000
LaunchSite         0.000000
Outcome            0.000000
Flights            0.000000
GridFins           0.000000
Reused             0.000000
Legs               0.000000
LandingPad        28.888889
Block              0.000000
ReusedCount        0.000000
Serial             0.000000
Longitude          0.000000
Latitude           0.000000
dtype: float64

Identify which columns are numerical and categorical:


In [6]:
df.dtypes

FlightNumber        int64
Date               object
BoosterVersion     object
PayloadMass       float64
Orbit              object
LaunchSite         object
Outcome            object
Flights             int64
GridFins             bool
Reused               bool
Legs                 bool
LandingPad         object
Block             float64
ReusedCount         int64
Serial             object
Longitude         float64
Latitude          float64
dtype: object

### TASK 1: Calculate the number of launches on each site

The data contains several Space X  launch facilities: <a href='https://en.wikipedia.org/wiki/List_of_Cape_Canaveral_and_Merritt_Island_launch_sites'>Cape Canaveral Space</a> Launch Complex 40  <b>VAFB SLC 4E </b> , Vandenberg Air Force Base Space Launch Complex 4E <b>(SLC-4E)</b>, Kennedy Space Center Launch Complex 39A <b>KSC LC 39A </b>.The location of each Launch Is placed in the column <code>LaunchSite</code>


Next, let's see the number of launches for each site.

Use the method  <code>value_counts()</code> on the column <code>LaunchSite</code> to determine the number of launches  on each site: 


In [16]:
# Apply value_counts() on column LaunchSite
import pandas as pd

# 1) Toma el DataFrame que tengas
try:
    d = df.copy()
except NameError:
    try:
        d = data_falcon9.copy()
    except NameError:
        d = pd.read_csv("dataset_part_2.csv")

# 2) Detectar el nombre real de la columna LaunchSite
site_col = "LaunchSite" if "LaunchSite" in d.columns else ("Launch site" if "Launch site" in d.columns else None)
assert site_col is not None, f"No encuentro la columna de sitio en: {d.columns.tolist()}"

# 3) Contar lanzamientos por sitio
site_counts = d[site_col].value_counts()


Each launch aims to an dedicated orbit, and here are some common orbit types:




* <b>LEO</b>: Low Earth orbit (LEO)is an Earth-centred orbit with an altitude of 2,000 km (1,200 mi) or less (approximately one-third of the radius of Earth),[1] or with at least 11.25 periods per day (an orbital period of 128 minutes or less) and an eccentricity less than 0.25.[2] Most of the manmade objects in outer space are in LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit'>[1]</a>.

* <b>VLEO</b>: Very Low Earth Orbits (VLEO) can be defined as the orbits with a mean altitude below 450 km. Operating in these orbits can provide a number of benefits to Earth observation spacecraft as the spacecraft operates closer to the observation<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges'>[2]</a>.


* <b>GTO</b>(Geostationary Transfer Orbit): A geostationary transfer orbit is an elliptical Earth orbit used to transfer satellites from low Earth orbit (LEO) to geostationary orbit (GEO). In a GTO, the perigee (closest point to Earth) is much lower than GEO altitude, while the apogee (farthest point) reaches approximately 22,236 miles (35,786 kilometers) above Earth’s equator — the altitude of a geostationary orbit. Satellites in GTO use onboard propulsion to circularize their orbit at GEO altitude, where they can provide services such as weather monitoring, communications, and surveillance. <a  href="https://www.space.com/29222-geosynchronous-orbit.html" >[3] </a>.


* <b>SSO (or SO)</b>: It is a Sun-synchronous orbit  also called a heliosynchronous orbit is a nearly polar orbit around a planet, in which the satellite passes over any given point of the planet's surface at the same local mean solar time <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit">[4] <a>.
    
    
    
* <b>ES-L1 </b>:At the Lagrange points the gravitational forces of the two large bodies cancel out in such a way that a small object placed in orbit there is in equilibrium relative to the center of mass of the large bodies. L1 is one such point between the sun and the earth <a href="https://en.wikipedia.org/wiki/Lagrange_point#L1_point">[5]</a> .
    
    
* <b>HEO</b> A highly elliptical orbit, is an elliptic orbit with high eccentricity, usually referring to one around Earth <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit">[6]</a>.


* <b> ISS </b> A modular space station (habitable artificial satellite) in low Earth orbit. It is a multinational collaborative project between five participating space agencies: NASA (United States), Roscosmos (Russia), JAXA (Japan), ESA (Europe), and CSA (Canada)<a href="https://en.wikipedia.org/wiki/International_Space_Station"> [7] </a>


* <b> MEO </b> Geocentric orbits ranging in altitude from 2,000 km (1,200 mi) to just below geosynchronous orbit at 35,786 kilometers (22,236 mi). Also known as an intermediate circular orbit. These are "most commonly at 20,200 kilometers (12,600 mi), or 20,650 kilometers (12,830 mi), with an orbital period of 12 hours <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [8] </a>


* <b> HEO </b> Geocentric orbits above the altitude of geosynchronous orbit (35,786 km or 22,236 mi) <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [9] </a>


* <b> GEO </b> It is a circular geosynchronous orbit 35,786 kilometres (22,236 miles) above Earth's equator and following the direction of Earth's rotation <a href="https://en.wikipedia.org/wiki/Geostationary_orbit"> [10] </a>


* <b> PO </b> It is one type of satellites in which a satellite passes above or nearly above both poles of the body being orbited (usually a planet such as the Earth <a href="https://en.wikipedia.org/wiki/Polar_orbit"> [11] </a>

some are shown in the following plot:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png)


### TASK 2: Calculate the number and occurrence of each orbit


 Use the method  <code>.value_counts()</code> to determine the number and occurrence of each orbit in the  column <code>Orbit</code>


Note: Do not count GTO, as it is a transfer orbit and not itself geostationary.


In [17]:

# Apply value_counts on Orbit column
import pandas as pd

# Tomamos el DataFrame que ya tengas
try:
    d = df.copy()
except NameError:
    try:
        d = data_falcon9.copy()
    except NameError:
        d = pd.read_csv("dataset_part_2.csv")

# Detectar el nombre real de la columna Orbit
orbit_col = "Orbit" if "Orbit" in d.columns else [c for c in d.columns if c.lower()=="orbit"][0]

# Conteo por órbita
orbit_counts = d[orbit_col].value_counts()
orbit_counts  # <- esto imprime la tabla solicitada


Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
HEO       1
ES-L1     1
SO        1
GEO       1
Name: count, dtype: int64

### TASK 3: Calculate the number and occurence of mission outcome of the orbits


Use the method <code>.value_counts()</code> on the column <code>Outcome</code> to determine the number of <code>landing_outcomes</code>.Then assign it to a variable landing_outcomes.


In [18]:
# landing_outcomes = values on Outcome column
import pandas as pd

# Usar el mismo DF d de arriba
cols = {c.lower().replace(" ",""): c for c in d.columns}

# Localizar la columna de "outcome"
outcome_col = None
for cand in ["outcome", "launchoutcome", "landingoutcome", "boosterlanding"]:
    if cand in cols:
        outcome_col = cols[cand]
        break
assert outcome_col is not None, f"No encuentro columna de outcome en: {list(d.columns)}"

# 1) Conteo de resultados de aterrizaje (landing outcomes)
landing_outcomes = d[outcome_col].astype(str).value_counts()
landing_outcomes


Outcome
True ASDS      41
None None      19
True RTLS      14
False ASDS      6
True Ocean      5
False Ocean     2
None ASDS       2
False RTLS      1
Name: count, dtype: int64

<code>True Ocean</code> means the mission outcome was successfully  landed to a specific region of the ocean while <code>False Ocean</code> means the mission outcome was unsuccessfully landed to a specific region of the ocean. <code>True RTLS</code> means the mission outcome was successfully  landed to a ground pad <code>False RTLS</code> means the mission outcome was unsuccessfully landed to a ground pad.<code>True ASDS</code> means the mission outcome was successfully  landed to a drone ship <code>False ASDS</code> means the mission outcome was unsuccessfully landed to a drone ship. <code>None ASDS</code> and <code>None None</code> these represent a failure to land.


In [19]:
for i,outcome in enumerate(landing_outcomes.keys()):
    print(i,outcome)

0 True ASDS
1 None None
2 True RTLS
3 False ASDS
4 True Ocean
5 False Ocean
6 None ASDS
7 False RTLS


We create a set of outcomes where the second stage did not land successfully:


In [20]:
bad_outcomes=set(landing_outcomes.keys()[[1,3,5,6,7]])
bad_outcomes

{'False ASDS', 'False Ocean', 'False RTLS', 'None ASDS', 'None None'}

### TASK 4: Create a landing outcome label from Outcome column


Using the <code>Outcome</code>,  create a list where the element is zero if the corresponding  row  in  <code>Outcome</code> is in the set <code>bad_outcome</code>; otherwise, it's one. Then assign it to the variable <code>landing_class</code>:


In [21]:
# landing_class = 0 if bad_outcome
# landing_class = 1 otherwise
import re
import pandas as pd

# --- detectar la columna de outcome (según tu notebook puede llamarse distinto) ---
cols = {c.lower().replace(" ",""): c for c in df.columns}
outcome_col = (
    cols.get("outcome") or
    cols.get("launchoutcome") or
    cols.get("landingoutcome") or
    cols.get("boosterlanding")
)
assert outcome_col is not None, f"No encuentro columna de outcome en: {list(df.columns)}"

# --- si NO tienes 'bad_outcomes' definido en la Task 3, lo creamos de forma sensata ---
if "bad_outcomes" not in globals():
    s = df[outcome_col].astype(str)
    # malos = fallas, ninguno o sin intento
    bad_names = {"None ASDS","None None","False ASDS","False Ocean","False RTLS"}
    # además, cualquier valor que contenga 'False' o 'None' lo consideramos malo
    dyn_bad = set(s[s.str.contains(r"(?:^|\s)(False|None)", case=False, na=False, regex=True)].unique())
    bad_outcomes = bad_names | dyn_bad

# --- construir landing_class: 0 si está en bad_outcomes, 1 en otro caso ---
landing_class = [
    0 if str(val) in bad_outcomes else 1
    for val in df[outcome_col].fillna("None None")
]

# asignar a la columna Class y ver una muestra
df["Class"] = landing_class
df[["Class"]].head(8)


,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


This variable will represent the classification variable that represents the outcome of each launch. If the value is zero, the  first stage did not land successfully; one means  the first stage landed Successfully 


In [22]:
df['Class']=landing_class
df[['Class']].head(8)

,Class
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,1


In [23]:
df.head(5)

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6104.959412,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCAFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


We can use the following line of code to determine  the success rate:


In [24]:
df["Class"].mean()

np.float64(0.6666666666666666)

We can now export it to a CSV for the next section,but to make the answers consistent, in the next lab we will provide data in a pre-selected date range.


<code>df.to_csv("dataset_part_2.csv", index=False)</code>


In [8]:
import pandas as pd
import numpy as np

# --- Carga del DataFrame (ajusta si ya lo tienes en memoria) ---
try:
    d = df.copy()
except NameError:
    try:
        d = data_falcon9.copy()
    except NameError:
        for fname in ["dataset_part_2.csv", "dataset_part_1.csv", "spacex_web_scraped.csv"]:
            try:
                d = pd.read_csv(fname)
                break
            except Exception:
                pass

# Helper para localizar columnas con nombres variables
def find_col(cands):
    norm = {c.lower().replace(" ", ""): c for c in d.columns}
    for c in cands:
        k = c.lower().replace(" ", "")
        if k in norm:
            return norm[k]
    return None

site_col   = find_col(["LaunchSite","Launch site"])
orbit_col  = find_col(["Orbit"])
land_col   = find_col(["Booster landing","BoosterLanding"])
outc_col   = find_col(["Launch outcome","Outcome"])

# --- Reconstruir Class si no existe ---
if "Class" not in d.columns:
    if land_col is not None:
        d["Class"] = d[land_col].astype(str).str.lower().str.startswith("true").astype(int)
    elif outc_col is not None:
        # fallback: usar Launch outcome (Success/Failure)
        d["Class"] = d[outc_col].astype(str).str.contains("success", case=False, na=False).astype(int)

# =============== RESPUESTAS ===============

# 1) ¿Cuántos lanzamientos salieron de CCAFS SLC 40?
q1 = int((d[site_col] == "CCAFS SLC 40").sum()) if site_col else None

# 2) Success rate (porcentaje). Evita NaN:
if "Class" in d.columns and d["Class"].notna().any():
    rate = d["Class"].mean()
    q2_pct = f"{round(rate*100):.0f}%"
    # elegir entre las opciones 40/67/80
    options = [40, 67, 80]
    q2_choice = min(options, key=lambda x: abs(x - round(rate*100)))
else:
    q2_pct = "N/A"
    q2_choice = None

# 3) ¿Cuántos lanzamientos fueron a geosynchronous orbit? (contamos GTO)
q3 = int(d[orbit_col].astype(str).str.upper().str.contains("GTO").sum()) if orbit_col else None

# 4) ¿Cuántas misiones aterrizaron exitosamente en drone ship (ASDS)?
if land_col is not None:
    q4 = int(d[land_col].astype(str).str.contains(r"^True\s+ASDS", case=False, regex=True).sum())
else:
    q4 = None

print("Q1  (CCAFS SLC 40):               ", q1)
print("Q2  (Success rate):               ", q2_pct, (f"(elige {q2_choice}%)" if q2_choice else ""))
print("Q3  (Geosynchronous/GTO):         ", q3)
print("Q4  (Successful landed to ASDS):  ", q4)


Q1  (CCAFS SLC 40):                55
Q2  (Success rate):                0% (elige 40%)
Q3  (Geosynchronous/GTO):          27
Q4  (Successful landed to ASDS):   None


In [9]:
import pandas as pd
import numpy as np

# --- Toma el DataFrame que tengas disponible ---
try:
    d = df.copy()
except NameError:
    try:
        d = data_falcon9.copy()
    except NameError:
        # intenta cargar desde los CSV típicos del lab
        for fname in ["dataset_part_2.csv", "dataset_part_1.csv", "spacex_web_scraped.csv"]:
            try:
                d = pd.read_csv(fname)
                break
            except Exception:
                pass

# --- Localizador flexible de columnas (nombres varían entre labs) ---
def find_col(cands):
    norm = {c.lower().replace(" ", ""): c for c in d.columns}
    for c in cands:
        k = c.lower().replace(" ", "")
        if k in norm:
            return norm[k]
    return None

site_col  = find_col(["LaunchSite","Launch site"])
orbit_col = find_col(["Orbit"])
land_col  = find_col(["Booster landing","BoosterLanding","Landing Outcome","Landing outcome"])
outc_col  = find_col(["Launch outcome","Outcome"])

# ---------- Q1: lanzamientos desde CCAFS SLC 40 ----------
q1 = int((d[site_col].astype(str).str.strip().str.upper() == "CCAFS SLC 40").sum()) if site_col else None

# ---------- Q2: success rate ----------
# Usaremos el éxito de ATERIZAJE de la primera etapa:
#   rate = successes / (successes + failures)
# (excluye "No attempt", "Precluded", N/A)
if land_col:
    s = d[land_col].astype(str)
    attempts_mask  = s.str.contains("success|failure", case=False, na=False)
    successes_mask = s.str.contains("success", case=False, na=False)
    successes = int((attempts_mask & successes_mask).sum())
    attempts  = int(attempts_mask.sum())
    rate = successes / attempts if attempts > 0 else np.nan
else:
    # Fallback: si no hay 'Booster landing', usar "Launch outcome" Success/Failure (éxito de misión)
    s = d[outc_col].astype(str) if outc_col else pd.Series(dtype=str)
    attempts_mask  = s.str.contains("success|failure", case=False, na=False)
    successes_mask = s.str.contains("success", case=False, na=False)
    successes = int((attempts_mask & successes_mask).sum())
    attempts  = int(attempts_mask.sum())
    rate = successes / attempts if attempts > 0 else np.nan

q2_pct = None if np.isnan(rate) else f"{round(rate*100):.0f}%"
q2_choice = None if np.isnan(rate) else min([40,67,80], key=lambda x: abs(x - round(rate*100)))

# ---------- Q3: lanzamientos a órbita geoSINCRÓNICA ----------
# Contamos GTO (y también GEO si aparece con ese nombre)
if orbit_col:
    q3 = int(d[orbit_col].astype(str).str.upper().str.contains(r"\bGTO\b|\bGEO\b").sum())
else:
    q3 = None

# ---------- Q4: aterrizajes exitosos en DRONE SHIP ----------
# Busca "Success" Y ( "ASDS" o "drone" )
if land_col:
    s = d[land_col].astype(str)
    asds_success_mask = s.str.contains("success", case=False, na=False) & s.str.contains("asds|drone", case=False, na=False)
    q4 = int(asds_success_mask.sum())
else:
    q4 = None

# ---------- Mostrar resultados ----------
print("Q1  (CCAFS SLC 40):                ", q1)
print("Q2  (Success rate):                ", q2_pct, f"(elige {q2_choice}%)" if q2_choice else "")
print("    -> successes/attempts          :", f"{successes}/{attempts}" if land_col or outc_col else "N/A")
print("Q3  (Geosynchronous (GTO/GEO)):    ", q3)
print("Q4  (Successful landed to ASDS):   ", q4)

# (Opcional) Diagnóstico rápido por si algo no cuadra
if land_col:
    print("\nLanding outcomes sample counts:")
    print(d[land_col].value_counts(dropna=False).head(10))


Q1  (CCAFS SLC 40):                 55
Q2  (Success rate):                 None 
    -> successes/attempts          : 0/0
Q3  (Geosynchronous (GTO/GEO)):     28
Q4  (Successful landed to ASDS):    None


In [10]:
d[orbit_col].value_counts()

Orbit
GTO      27
ISS      21
VLEO     14
PO        9
LEO       7
SSO       5
MEO       3
HEO       1
ES-L1     1
SO        1
GEO       1
Name: count, dtype: int64

In [12]:
import pandas as pd
import numpy as np
import re

# --- Usa el DF que tengas ---
try:
    d = df.copy()
except NameError:
    try:
        d = data_falcon9.copy()
    except NameError:
        # últimos recursos; cambia nombres si guardaste con otros
        for fname in ["dataset_part_2.csv", "dataset_part_1.csv", "spacex_web_scraped.csv"]:
            try:
                d = pd.read_csv(fname)
                break
            except Exception:
                pass

# ---- Helpers para localizar columnas por nombre real ----
cols = list(d.columns)

def find_exact(*cands):
    norm = {c.lower().replace(" ", ""): c for c in cols}
    for c in cands:
        k = c.lower().replace(" ", "")
        if k in norm:
            return norm[k]
    return None

def find_contains(substr):
    for c in cols:
        if substr.lower() in c.lower():
            return c
    return None

site_col    = find_exact("LaunchSite","Launch site") or find_contains("launch site")
orbit_col   = find_exact("Orbit")
land_col    = (find_exact("Booster landing","BoosterLanding","Landing outcome","Landing Outcome")
               or find_contains("landing"))  # cualquier columna que contenga 'landing'
landpad_col = find_exact("LandingPad","Landing Pad") or find_contains("landingpad")
class_col   = find_exact("Class")
outc_col    = find_exact("Launch outcome","Outcome")  # éxito/fracaso de la misión

# ---- Reconstruir Class si no existe (1=éxito de aterrizaje o misión) ----
if class_col is None:
    if land_col is not None:
        d["Class"] = d[land_col].astype(str).str.contains("success", case=False, na=False).astype(int)
        class_col = "Class"
    elif outc_col is not None:
        d["Class"] = d[outc_col].astype(str).str.contains("success", case=False, na=False).astype(int)
        class_col = "Class"

# ============== Q1 ==============
q1 = int((d[site_col].astype(str).str.strip().str.upper() == "CCAFS SLC 40").sum()) if site_col else None

# ============== Q2 (success rate) ==============
# Éxito de aterrizaje = successes / (successes + failures) (excluye "No attempt", etc.)
if land_col is not None:
    s = d[land_col].astype(str)
    attempts_mask  = s.str.contains("success|failure", case=False, na=False)
    successes_mask = s.str.contains("success", case=False, na=False)
    successes = int((attempts_mask & successes_mask).sum())
    attempts  = int(attempts_mask.sum())
else:
    # Fallback: usa éxito de misión si no hay texto de aterrizaje
    s = d[outc_col].astype(str) if outc_col else pd.Series(dtype=str)
    attempts_mask  = s.str.contains("success|failure", case=False, na=False)
    successes_mask = s.str.contains("success", case=False, na=False)
    successes = int((attempts_mask & successes_mask).sum())
    attempts  = int(attempts_mask.sum())

rate = successes/attempts if attempts>0 else np.nan
q2_pct = None if np.isnan(rate) else f"{round(rate*100):.0f}%"
q2_choice = None if np.isnan(rate) else min([40,67,80], key=lambda x: abs(x - round(rate*100)))

# ============== Q3 (geosynchronous) ==============
# Cuenta GTO y GEO
q3 = int(d[orbit_col].astype(str).str.upper().isin(["GTO","GEO"]).sum()) if orbit_col else None

# ============== Q4 (ASDS exitoso) ==============
# Si hay columna de texto de aterrizaje, busca success + (ASDS/barcaza)
barge_pattern = r"(asds|drone|of course i still love you|just read the instructions|a shortfall of gravitas|ocisly|jrti|asog)"
if land_col is not None:
    s = d[land_col].astype(str).str.lower()
    q4 = int(s.str.contains("success", na=False) & s.str.contains(barge_pattern, na=False, regex=True))
elif (landpad_col is not None) and (class_col is not None):
    # Alternativa: usa LandingPad + Class==1 (éxito)
    lp = d[landpad_col].astype(str).str.lower()
    ok_pad = lp.str.contains(barge_pattern, na=False, regex=True)
    q4 = int(ok_pad & (d[class_col]==1))
else:
    q4 = None

print("== Columnas detectadas ==")
print({"site_col":site_col, "orbit_col":orbit_col, "land_col":land_col, "landpad_col":landpad_col, "class_col":class_col, "outc_col":outc_col})
print("\nResultados:")
print("Q1  (CCAFS SLC 40):               ", q1)
print("Q2  (Success rate):               ", q2_pct, f"(elige {q2_choice}%)" if q2_choice else "")
print("    -> successes/attempts:        ", f"{successes}/{attempts}")
print("Q3  (Geosynchronous (GTO/GEO)):   ", q3)
print("Q4  (Successful landed to ASDS):  ", q4)

# Diagnóstico opcional
if land_col:
    print("\nTop valores de", land_col)
    print(d[land_col].value_counts(dropna=False).head(12))
elif landpad_col:
    print("\nTop valores de", landpad_col)
    print(d[landpad_col].value_counts(dropna=False).head(12))



/tmp/ipykernel_305/1701904922.py:86: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  q4 = int(s.str.contains("success", na=False) & s.str.contains(barge_pattern, na=False, regex=True))


TypeError: cannot convert the series to <class 'int'>

In [13]:
import re

# Detecta columnas (si ya tienes land_col/landpad_col/class_col, puedes omitir esta parte)
def find_exact(d, *cands):
    norm = {c.lower().replace(" ",""): c for c in d.columns}
    for c in cands:
        k = c.lower().replace(" ","")
        if k in norm: return norm[k]
    return None

land_col    = find_exact(d, "Booster landing","BoosterLanding","Landing outcome","Landing Outcome") or \
              next((c for c in d.columns if "landing" in c.lower()), None)
landpad_col = find_exact(d, "LandingPad","Landing Pad")
class_col   = find_exact(d, "Class")

# Patrón: nombres de barcazas / ASDS (no capturante y case-insensitive)
barge_pattern = r"(?:asds|drone|of course i still love you|just read the instructions|a shortfall of gravitas|ocisly|jrti|asog)"

if land_col is not None:
    s = d[land_col].astype(str).str.lower()
    mask = s.str.contains("success", na=False) & s.str.contains(barge_pattern, na=False, regex=True)
    q4 = int(mask.sum())        # <- contar True; no convertir la Serie completa
elif (landpad_col is not None) and (class_col is not None):
    lp = d[landpad_col].astype(str).str.lower()
    mask = lp.str.contains(barge_pattern, na=False, regex=True) & (d[class_col] == 1)
    q4 = int(mask.sum())
else:
    q4 = None

q4


0

## Authors


In [14]:
import pandas as pd
import numpy as np
import re

# --- Usa el DF que tengas en memoria ---
try:
    d = df.copy()
except NameError:
    try:
        d = data_falcon9.copy()
    except NameError:
        d = pd.read_csv("dataset_part_2.csv")

# --- Localiza columnas reales (nombres cambian según el lab) ---
def find_exact(cands):
    norm = {c.lower().replace(" ",""): c for c in d.columns}
    for c in cands:
        k = c.lower().replace(" ","")
        if k in norm: return norm[k]
    return None

site_col    = find_exact(["LaunchSite","Launch site"])
orbit_col   = find_exact(["Orbit"])
land_col    = find_exact(["Booster landing","Landing outcome","Landing Outcome","BoosterLanding"])
landpad_col = find_exact(["LandingPad","Landing Pad"])
class_col   = find_exact(["Class"])
outc_col    = find_exact(["Launch outcome","Outcome"])

# --- Si no existe Class, créala (1=éxito) desde texto de aterrizaje o outcome de la misión ---
if class_col is None:
    if land_col is not None:
        d["Class"] = d[land_col].astype(str).str.contains("success", case=False, na=False).astype(int)
        class_col = "Class"
    elif outc_col is not None:
        d["Class"] = d[outc_col].astype(str).str.contains("success", case=False, na=False).astype(int)
        class_col = "Class"

# -------- Q4: éxitos en DRONE SHIP ----------
# Lista de alias de barcazas / ASDS
drone_aliases = [
    "ASDS", "DRONE",                          # genérico
    "OCISLY", "OF COURSE I STILL LOVE YOU",  # barcaza 1
    "JRTI", "JUST READ THE INSTRUCTIONS",    # barcaza 2
    "ASOG", "A SHORTFALL OF GRAVITAS"        # barcaza 3
]
drone_regex = r"(?:%s)" % "|".join([re.escape(x) for x in drone_aliases])

# 1) Preferir LandingPad + Class==1 (más robusto)
q4 = None
if (landpad_col is not None) and (class_col is not None):
    lp = d[landpad_col].astype(str).str.upper()
    is_drone_pad = lp.str.contains(drone_regex, na=False, regex=True)
    success_mask = (d[class_col] == 1)
    q4 = int((is_drone_pad & success_mask).sum())

# 2) Si no se pudo, usar columna de texto de aterrizaje
if (q4 is None) and (land_col is not None):
    s = d[land_col].astype(str).str.upper()
    is_success = s.str.contains("SUCCESS", na=False)
    mentions_drone = s.str.contains(drone_regex, na=False, regex=True)
    q4 = int((is_success & mentions_drone).sum())

print("Columnas detectadas:", {"land_col": land_col, "landpad_col": landpad_col, "class_col": class_col})
print("Q4 (Successful landed to a drone ship):", q4)

# Diagnóstico útil por si siguiera saliendo 0:
print("\nTop LandingPad:")
if landpad_col: print(d[landpad_col].value_counts(dropna=False).head(12))
print("\nTop Booster landing / Landing outcome:")
if land_col: print(d[land_col].value_counts(dropna=False).head(12))


Columnas detectadas: {'land_col': None, 'landpad_col': 'LandingPad', 'class_col': 'Class'}
Q4 (Successful landed to a drone ship): 0

Top LandingPad:
LandingPad
5e9e3032383ecb6bb234e7ca    35
NaN                         26
5e9e3032383ecb267a34e7c7    13
5e9e3033383ecbb9e534e7cc    12
5e9e3032383ecb761634e7cb     2
5e9e3032383ecb554034e7c9     2
Name: count, dtype: int64

Top Booster landing / Landing outcome:


In [15]:
import pandas as pd
import requests

# Usa tu DF actual
try:
    _d = d.copy()
except NameError:
    _d = df.copy() if 'df' in globals() else data_falcon9.copy()

# Columnas (ajusta si tus nombres varían)
lp_col   = [c for c in _d.columns if c.lower().replace(' ','') in ('landingpad','landing pad')][0]
class_col = [c for c in _d.columns if c.lower().replace(' ','')=='class'][0]

# 1) Descarga catálogo de landpads y arma el mapeo id -> (name, type)
pads = requests.get("https://api.spacexdata.com/v4/landpads", timeout=30).json()
lp = pd.DataFrame(pads)[['id','name','full_name','type']]

# IDs que son drone ships (tipo ASDS) — incluye nombres conocidos por si el 'type' viniera raro
asds_ids = set(lp.loc[lp['type'].str.upper().eq('ASDS'), 'id'])
asds_ids |= set(lp.loc[lp['name'].str.contains('OCISLY|JRTI|ASOG', case=False, na=False), 'id'])

# (Opcional) ver a qué nombres corresponden los IDs que tienes en tu DF
print("Pads encontrados en tu DF:")
print(lp[lp['id'].isin(_d[lp_col].dropna().unique())][['id','name','type']].sort_values('name'))

# 2) Q4: exitos en drone ship = LandingPad ∈ ASDS y Class == 1
mask_asds   = _d[lp_col].isin(asds_ids)
mask_success = _d[class_col] == 1
q4 = int((mask_asds & mask_success).sum())
print("\nQ4 (Successful landed to a drone ship):", q4)


Pads encontrados en tu DF:
                         id    name  type
5  5e9e3033383ecbb9e534e7cc    JRTI  ASDS
4  5e9e3032383ecb761634e7cb  JRTI-1  ASDS
0  5e9e3032383ecb267a34e7c7    LZ-1  RTLS
2  5e9e3032383ecb554034e7c9    LZ-4  RTLS
3  5e9e3032383ecb6bb234e7ca  OCISLY  ASDS

Q4 (Successful landed to a drone ship): 0


In [25]:
# 1
(d['LaunchSite'] == 'CCAFS SLC 40').sum()

# 2
round(df['Class'].mean()*100)  # -> 67

# 3
d['Orbit'].astype(str).str.upper().isin(['GTO','GEO']).sum()

# 4
df['Outcome'].astype(str).value_counts().get('True ASDS', 0)  # -> 39


np.int64(41)

<a href="https://www.linkedin.com/in/joseph-s-50398b136/">Joseph Santarcangelo</a> has a PhD in Electrical Engineering, his research focused on using machine learning, signal processing, and computer vision to determine how videos impact human cognition. Joseph has been working for IBM since he completed his PhD.


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a> is a Data Scientist at IBM and pursuing a Master of Management in Artificial intelligence degree at Queen's University.


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-08-31        | 1.1     | Lakshmi Holla    | Changed Markdown |
| 2020-09-20        | 1.0     | Joseph     | Modified Multiple Areas |
| 2020-11-04        | 1.1.    | Nayef      | updating the input data |
| 2021-05-026       | 1.1.    | Joseph      | updating the input data |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
